# COLAB: Training a CNN to Detect Cats and Dogs

My computer is unable to train any neural net due to limits of computational power. This notebok is altered

In [24]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [28]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os

### Create Directories to Store Data

In [29]:
try:
    base_dir = 'data'

    os.mkdir(base_dir)
    os.mkdir(base_dir + '/training')
    os.mkdir(base_dir + '/validation')
    os.mkdir(base_dir + '/testing')
    os.mkdir(base_dir + '/training/cats')
    os.mkdir(base_dir + '/training/dogs')
    os.mkdir(base_dir + '/validation/cats')
    os.mkdir(base_dir + '/validation/dogs')
    os.mkdir(base_dir + '/testing/cats')
    os.mkdir(base_dir + '/testing/dogs')
except FileExistsError:
    pass

### Create function to sperate data

Split the data into training, validation, and testing directories to use TensorFlow`flow_from_directory`

In [30]:
import random
from shutil import copyfile

In [42]:
def sort_cats_dogs(data):
    dogs = []
    cats = []
    for datum in data:
        if 'dog' in datum:
            dogs.append(datum)

        if 'cat' in datum:
            cats.append(datum)

    return cats, dogs

def fill_directories(data, base, training, validation, split_size):
    train = random.sample(data, round(split_size * len(data)))
    valid = list(set(data) - set(train))

    for x in train:
         copyfile(base + x, training + x)

    for x in valid:
         copyfile(base + x, validation + x)

def make_validation(base, training, validation, split_size=0.9):
    """
    base: source where data is coming from
    training, validation: relative paths to respecitve directories
    split_size: proportion of data to keep in training set
    """
            
    # Get all files into array and get rid of files that are empty
    data = [x for x in os.listdir(base) if os.path.getsize(base + x) > 0]
    
    # Split into cats and dogs
    cats, dogs = sort_cats_dogs(data)
    
    # Fill directories
    fill_directories(cats, base, training+'/cats/', validation+'/cats/', split_size)
    fill_directories(dogs, base, training+'/dogs/', validation+'/dogs/', split_size)
    
def make_test(source, target):
    data = [x for x in os.listdir(source) if os.path.getsize(source + x) > 0]

    cats, dogs = sort_cats_dogs(data)
    print(len(cats))
    print(len(dogs))
    return 
    
    for cat in cats:
        copyfile(source + cat, target + '/cats/')
    
    for dog in dogs:
        copyfile(source + dog, target + '/dogs/')

In [34]:
training = 'data/training'
validation = 'data/validation'
source = 'train/'

make_validation(source, training, validation)

In [43]:
testing = 'data/testing'
source = 'test1/'

make_test(source, testing)

0
0


### Define ```ImageDataGenerator```

In [38]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [39]:
training = 'data/training/'
validation = 'data/validation/'
testing = 'data/testing/'

train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    training,
    target_size=(150,150),
    batch_size=10,
    class_mode='binary'
)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    validation,
    target_size=(150,150),
    batch_size=10,
    class_mode='binary'
)

testing_datagen = ImageDataGenerator(rescale=1./255)
testing_datagen = testing_datagen.flow_from_directory(
    testing,
    target_size=(150,150),
    batch_size=10,
    class_mode='binary'
)

Found 22500 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


### Define the CNN

In [44]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(input_shape=(150,150,3)),
    tf.keras.layers.Dense(150, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [45]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 16)        2320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        1

### Train Model

In [46]:
history = model.fit(train_generator, validation_data=validation_generator, epochs=50)

Epoch 1/50


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/conv2d/Conv2D (defined at <ipython-input-46-a207bbc5f60e>:1) ]] [Op:__inference_train_function_1040]

Function call stack:
train_function
